# Gathering Django Contributors

---
<font color='red'><b>Important</b></font>: This Notebook relies on a [now deprecated API](https://github.com/django/code.djangoproject.com/pull/198#issuecomment-2398110549). That API was deprecated due to infrequent use, but also it would [leak clear email addresses](https://github.com/glasnt/dceu2022-sprints-work/issues/3). Results from previous executions of this notebook have been removed.  

---

Skip to: [Scope](#scope), [Assumptions](#assumptions), [Data Gathering](#data-gathering), [Analysis](#analysis), [Results](#results).

---

Who contributed to Django 4.1?

This answer should be easy to get, surely, because Django is hosted on GitHub. 

Well, yes, but more goes into a release of Django than just GitHub. There's also the separate ticketing system and translations, just to name two of the major systems involved. 

The Django repo also uses an interesting way of cutting releases, as described in [Carlton's notes](https://noumenal.es/posts/what-is-django-4/zj2/) from when he tried to answer this question. 

To get a more complete view of the information, we need to get data from a few systems. 

To start: which version do we want to get the information for? In this case, 4.1. (Noting of course that this data may still be in flux as this is still the current minor version, and will still have data changes until it is EOL'd some time in 2023 or there abouts.)

There's also the information just in the comments themselves: 

* The author and the committer (not always the same person)
* The Trac issue (often in the comment message itself)
* Any manual thanks (often in the case of security fixes) as part of the message
* Any discussions in attached pull requests. 

And from there, the Trac data, particularly the discussions in the ticket itself. 

So let's dive in.

_Along the way there will also be italiced notes that mostly serve as "here's how I did the thing" (I'm very new to notebooks!)_

---


## Defining Scope  <a class="anchor" id="scope"></a>

Before we can start analysing, we need to gather the data. 

Based on the original blog post, we first need to determine the start and stop commit range for the branch in question. 

To do that, let's get setup: 

 * Clone a local copy of the django repo
   * _this command uses `!` to call out to `git` in the shell_

In [1]:
!git clone https://github.com/django/django django-codebase

fatal: destination path 'django-codebase' already exists and is not an empty directory.


 * Install the `gitPython` package
   * _this command uses `%%capture%%` as not to print the possibly lengthly output to the notebook_


In [2]:
%%capture
import sys
!{sys.executable} -m pip install GitPython

 * and set up for analysis

In [3]:
from git import Repo
repo = Repo("django-codebase")

We want to target the 4.1 release: 

In [4]:
target_release = "4.1"
previous_release = "4.0"  # semver -1

So using Carlton's method, let's get the start and end commits, and the merge commit: 

In [5]:
start_commit = repo.commit(previous_release)
end_commit = repo.commit(target_release)
merge_base = repo.merge_base(start_commit, end_commit)[0]

commits = list(repo.iter_commits(str(merge_base) + ".." + str(end_commit)))

We can check the start and end commits to confirm we're in the right range, and about the right commit number: 

In [6]:
from datetime import datetime

for commit in [start_commit, end_commit]: 
    print(datetime.fromtimestamp(commit.authored_date), commit.author, "\n" + commit.message)
    
print("Commit count:", len(commits))

2021-12-07 20:07:32 Mariusz Felisiak 
[4.0.x] Bumped version for 4.0 release.

2022-08-03 18:33:01 Carlton Gibson 
[4.1.x] Bumped version for 4.0 release.

Commit count: 829


This looks about right: a ~8 month development window, bookmarked by Django Fellows. 

---


# Assumptions <a class="anchor" id="assumptions"></a>



One of the assumptions we're making here is that we don't just want a list of the authors of the commits. 

We could already get this data, very easily. 

But more work goes into releasing open source software then just those who author code. 

Django uses a separate issue tracking system, [Trac](https://code.djangoproject.com/), hosted on https://code.djangoproject.com/. 

Django also uses a separate system for handling translations of the documentation, [Transifex](https://www.transifex.com/django/), which are output to https://github.com/django/django-docs-translations. (Documentation itself is in the main repo.)

For our purposes, here are our assumptions: 


Commits: 

* The commits in the release branch are the scope of the changes that we will focus on for a release. 
* The authors of those commits are contributors. 

Associating Tickets: 

* For any commit in scope, the linked tickets are associated work. 
* The state of those tickets is not in question, because if work happened to make a commit, it counts. 

Tickets: 

* Any non-trivial interaction on any linked ticket is a candidate for inclusion, where activity on a ticket will be further analysed. 
  
Associating Pull Requests: 

* Where tickets have linked GitHub Pull Requests, the linked pull requests are associated work. 
* Those pull requests may have additional interactions (e.g. code review, comments). 

Pull Requests: 

* Any non-trivial interaction on any linked pull request is a candidate for inclusion, where activity on a pull request will be further analysed. 

Translations: 
 
* All contributed translations are associated work. 

**Note**: In this case "non-trivial" interactions are those interactions that are work done. Arguments can be made about how much interactions count. The quantitative number in this case doesn't count (one interaction of high quality could count, many spam interactions could not). These will be analysed later. 

--

## Gathering Data

### From `git` 

With our local git clone, we can go through all the commits in our scope and gather the information about the associated tickets. 

We're using the assumption that ticket numbers appear in git messages. 

In [8]:
import re 
def get_git_commits(commit):
    git_commits.append(
        {
            "django_version": target_release,
            "commit_sha": commit.hexsha,
            "datetime": commit.authored_date,
            "author": commit.author.name,
            "author_email": commit.author.email,
            "committer": commit.committer.name,
            "committer_email": commit.committer.email,
            "message": commit.message,
        }
    )

    # Get all ticket references in message
    # NOTE(glasnt): this will include "Fixed", but also "Refs" (which may include older tickets)
    tickets = [x.replace("#", "") for x in re.findall("\#[0-9]*", commit.message)]

    for ticket in tickets:
        if ticket:
            git_trac_links.append(
                {"commit_sha": commit.hexsha, "trac_ticket_id": ticket}
            )

            
git_commits = []
git_trac_links = []
tickets = [] 
            
for commit in commits:
    get_git_commits(commit)
    
# Get unique list
tickets = list(set([k["trac_ticket_id"] for k in git_trac_links]))

print("Git Commits:", len(git_commits))
print("Git Trac Links:", len(git_trac_links))
print("Tickets:", len(tickets))

Git Commits: 829
Git Trac Links: 563
Tickets: 402


So we now have our associated tickets. 

---

### From Trac
From here, we need to get the information for each of these tickets out of Trac. Since this operation is expensive, we'll [cache the results](https://realpython.com/caching-external-api-requests/). 

In [10]:
%%capture
import sys
!{sys.executable} -m pip install requests "requests-cache[all]" tqdm ipywidgets

In [11]:
import requests
from requests_cache import CachedSession

# Note: POST may be ignored by default! So ensure we cache that. 
session = CachedSession('api_cache', backend='sqlite', allowable_methods=('GET', 'POST'))
# Note: this will create a api_cache.sqlite file, which will contain your GitHub token (later).
# Don't commit this file!

With that setup, we'll now go through all the trac tickets we found: 

In [13]:
import json

# Shout out to John Sandall https://twitter.com/John_Sandall/status/1573711570894462977 
from tqdm.notebook import tqdm, trange

def get_trac_details(ticket_no):
    """For a ticket number, get the ticket information and changes from Trac"""
    
    ticket_comments = []
    
    # Shout out to rixx https://gist.github.com/rixx/422392d2aa580b5d286e585418bf6915 
    resp = session.post(
        DJANGO_TRAC,
        data=json.dumps(
            {"method": "ticket.get", "id": ticket_no, "params": [ticket_no]}
        ),
        headers={"Content-Type": "application/json"},
    )

    data = resp.json()["result"][3]
    
    ticket = {
        "ticket_id": ticket_no,
        "status": data["status"],
        "reporter": data["reporter"],
        "resolution": data["resolution"],
        "description": data["description"],
    }

    # struct ticket.changeLog(int id, int when=0)
    # Return the changelog as a list of tuples of the form
    # (time, author, field, oldvalue, newvalue, permanent).
    response = session.post(
        DJANGO_TRAC,
        data=json.dumps(
            {"method": "ticket.changeLog", "id": ticket_no, "params": [ticket_no]}
        ),
        headers={"Content-Type": "application/json"},
    )
    
    changes = response.json()["result"]

    for change in changes:
        name = change[1]
        if name:
            name = name.split("<")[0].strip() # remove emails,
        ticket_comments.append(
            {
                "ticket_id": ticket_no,
                "datetime": change[0]["__jsonclass__"][1],
                "name": name,
                "change_type": change[2],
                "old_value": change[3],
                "new_value": change[4],
            }
        )
    return ticket, ticket_comments



trac_tickets = []
trac_ticket_comments = []

DJANGO_TRAC = "https://code.djangoproject.com/jsonrpc"

for ticket_no in tqdm(tickets): 
    ticket, ticket_comments = get_trac_details(ticket_no)
    trac_tickets.append(ticket)
    trac_ticket_comments += ticket_comments

print("Trac Tickets:", len(trac_tickets))
print("Trac Ticket Comments:", len(trac_ticket_comments))

  0%|          | 0/402 [00:00<?, ?it/s]

Trac Tickets: 402
Trac Ticket Comments: 9799


We now have a list of all the Trac tickets and comments associated to commits in our target release. 

----

### From GitHub

From here, we can check for additional interactions on GitHub. 

Django's trac uses a [custom patch](https://github.com/django/code.djangoproject.com/blob/main/trac-env/htdocs/tickethacks.js#L38) to use the GitHub API to search for Pull Requests with the linked tickets, much like we did for associating commits to track tickets. 

We can use this same GitHub Search API to get the list of associated pull requests, then all the interactions on those pull requests. 

_Jupyter notebooks support `.env` files! Use the config below to load your env_

In [16]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [17]:
# Use a GitHub token to get better rate limits!
import os
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")

In [18]:
# Check your rate limits manually

# https://docs.github.com/en/rest/rate-limit 
# curl \
#  -H "Accept: application/vnd.github+json" \
#  -H "Authorization: Bearer <YOUR-TOKEN>" \
#  https://api.github.com/rate_limit

from datetime import datetime
from pprint import pprint

with session.cache_disabled():
    resp = session.get("https://api.github.com/rate_limit",
            headers={
                "Authorization": f"Bearer {GITHUB_TOKEN}",
                "Accept": "application/vnd.github.v3.raw",
            }
        )
if "message" in resp.json().keys():
    print(resp.json())
else: 
    data = resp.json()["resources"]
    now = datetime.now()

    for limit_type in ["core", "search"]: 
        d = data[limit_type]
        print(f"GitHub {limit_type} API limit:", d["used"], "/", d["limit"], ", resets", datetime.fromtimestamp(d["reset"]) )


GitHub core API limit: 0 / 5000 , resets 2022-11-24 13:37:55
GitHub search API limit: 0 / 30 , resets 2022-11-24 12:38:55


In [22]:
import time 

def github_api(uri):
    """For a GitHub API path, return the JSON data"""
    resp = session.get(
        "https://api.github.com" + uri,
        headers={
            "Authorization": f"Bearer {GITHUB_TOKEN}",
            "Accept": "application/vnd.github+json",
        }
    )

    if resp.status_code != 200:
        if "X-RateLimit-Remaining" in resp.headers and resp.headers["X-RateLimit-Remaining"] == 0:
            wait_seconds = int(resp.headers.get("x-ratelimit-reset")) - int(time.time())
            wait_minutes = int(wait_seconds / 60)
            print(f"Rate limit expired. Wait {wait_minutes} minutes (or {wait_seconds} seconds).")
        raise ValueError(resp.json()["message"])

    return resp.json()

def search_for_pull_requests(ticket_id):
    """For a ticket ID, search for related pull requests,
    based on the system trac uses"""
    data = github_api(
        "/search/issues?q=repo:django/django+in:title+type:pr+"
        + "%23" + ticket_id + "%20" 
        + "+%23"+ ticket_id + "%2C" 
        + "+%23"+ ticket_id + "%3A" 
        + "+%23"+ ticket_id + "%29"
    )["items"]

    return [x["number"] for x in data]


def get_pull_request(pr_id): 
    """For a given pull request ID, get the minimum data from the github API """
    data = github_api(f"/repos/django/django/pulls/{pr_id}")
    
    pr = {"id": data["number"], 
          "state": data["state"], 
          "user": data["user"]["login"], 
         }
    
    return pr

pull_request_ids = []
pull_requests = []

for ticket_no in tqdm(tickets): 
    pull_request_ids += search_for_pull_requests(ticket_no)

pull_request_ids = list(set(pull_request_ids))

for pr_id in tqdm(pull_request_ids):
    pull_requests.append(get_pull_request(pr_id))

print("Pull requests:", len(pull_requests))

  0%|          | 0/402 [00:00<?, ?it/s]

  0%|          | 0/756 [00:00<?, ?it/s]

Pull requests: 756


For all these pull requests, we need to get the comments associated.  

In [23]:
def get_comments_from_pull_request(pull_request_id):
    """For a pull request ID, get the comments"""
    comments = []

    # Comments
    data = github_api(f"/repos/django/django/pulls/{pull_request_id}/comments")

    for record in data:
        comments.append(
            {
                "user": record["user"]["login"],
                "commit_id": record["commit_id"],
                "message": record["body"],
                "pull_request": pull_request_id
            }
        )

    # Review Comments
    data = github_api(f"/repos/django/django/issues/{pull_request_id}/comments")

    for record in data:
        comments.append(
            {
                "user": record["user"]["login"],
                "commit_id": None,
                "message": record["body"],
                "pull_request": pull_request_id,
            }
        )

    return comments

pr_comments = []

for request in tqdm(pull_requests):
    pr_comments += get_comments_from_pull_request(request["id"])

print("Pull Request Comments:", len(pr_comments))


  0%|          | 0/756 [00:00<?, ?it/s]

Pull Request Comments: 6263


### Security Thanks

Another dataset we need to add in is security acknowledgements. 

Security reports are [separate to tickets](https://docs.djangoproject.com/en/dev/internals/security/), and thus don't follow the normal git flow. They are handled by the security team, and [credit is given separately](https://twitter.com/carltongibson/status/1588455611049676800).

So, we need to manually search for that credit. 

_This section uses a little bit of pandas, where I was trying out the interface. The output is also very nicely compact here, but it doesn't get used much later on._

In [ ]:
import pandas as pd

git_commits_df = pd.DataFrame(git_commits)
pd.set_option('display.max_colwidth', None) # ensure all data is displayed. 
git_commits_df[git_commits_df.message.str.contains("Thanks")][["author", "message"]]


From here, let's isolate those thanks mentions (shout out to [https://pythex.org/](https://pythex.org/)!)

In [25]:
git_commits_df.message.str.extract(r'Thanks ([a-zA-Z ]+) for').dropna()

,0
3,to Motoyasu Saburi
6,to Paolo Melchiorre
7,to Claude Paroz
14,Johnny Metz
24,Claude Paroz
...,...
744,Lucidot for the report and Mariusz Felisiak
779,Simon Charette
791,Simon Charette
797,to Claude Paroz


The problem you're seeing here is that we're dropping records: 

In [26]:
contains_records = len(git_commits_df[git_commits_df.message.str.contains("Thanks")][["author", "message"]])
extracts_records = len(git_commits_df.message.str.extract(r'Thanks ([a-zA-Z ]+) for').dropna())

print("Compare: contains has", contains_records, "and extracts has", extracts_records)
print("Do they match?", contains_records == extracts_records)

Compare: contains has 83 and extracts has 67
Do they match? False


An issue here is that there are some users with non-alpha characters in their names. 

In [27]:
example_user = "Michal"
example_commits = git_commits_df[git_commits_df.message.str.contains(example_user).dropna()]["message"]
print(example_commits)

778    Fixed #33159 -- Reverted "Fixed #32970 -- Changed WhereNode.clone() to create a shallow copy of children."\n\nThis reverts commit e441847ecae99dd1ccd0d9ce76dbcff51afa863c.\r\n\r\nA shallow copy is not enough because querysets can be reused and\r\nevaluated in nested nodes, which shouldn't mutate JOIN aliases.\r\n\r\nThanks Michal Čihař for the report.
Name: message, dtype: object


So we need to make sure we're using better regex.

In [31]:
w_filter = git_commits_df.message.str.extract(r'Thanks ([\w ]+) for').dropna()
print(w_filter)

print("Compare: contains has", contains_records, "and extracts has", len(w_filter))
print("Do they match?", contains_records == len(w_filter))

                       0
3     to Motoyasu Saburi
6    to Paolo Melchiorre
7        to Claude Paroz
14           Johnny Metz
24          Claude Paroz
..                   ...
778         Michal Čihař
779       Simon Charette
791       Simon Charette
797      to Claude Paroz
809      Benjamin Locher

[74 rows x 1 columns]
Compare: contains has 83 and extracts has 74
Do they match? False


But even then we're missing some! Such as: 



In [32]:
git_commits_df[git_commits_df.message.str.contains("Splunk")][["author", "message"]]

,author,message
202,Mariusz Felisiak,"Fixed CVE-2022-28346 -- Protected QuerySet.annotate(), aggregate(), and extra() against SQL injection in column aliases.\n\nThanks Splunk team: Preston Elder, Jacob Davis, Jacob Moore,\nMatt Hanson, David Briggs, and a security researcher: Danylo Dmytriiev\n(DDV_UA) for the report.\n"


So let's just open it up to any character. 

In [34]:
dot_filter = git_commits_df.message.str.extract(r'Thanks (.*) for').dropna()

print(dot_filter)

print("Compare: contains has", contains_records, "and extracts has", len(dot_filter))
print("Do they match?", contains_records == len(dot_filter))

                       0
3     to Motoyasu Saburi
6    to Paolo Melchiorre
7        to Claude Paroz
14           Johnny Metz
24          Claude Paroz
..                   ...
778         Michal Čihař
779       Simon Charette
791       Simon Charette
797      to Claude Paroz
809      Benjamin Locher

[80 rows x 1 columns]
Compare: contains has 83 and extracts has 80
Do they match? False


But then we're still missing some!

😨

So let's just do it manually. 

In [38]:
def flatten(l): 
    return [item for sublist in l for item in sublist]

def unique(l): 
    l = list(filter(lambda i: i is not None, l))
    l = [a.strip() for a in l]
    return sorted(list(set(l)), key=str.casefold)

thanks_messages = git_commits_df[git_commits_df.message.str.contains("Thanks")]["message"].values.tolist()

thanks = []
security_thanks = []
second_pass = []

for msg in thanks_messages: 
    found = False
    thanks = re.findall("Thanks (.*) for", msg)
    if not thanks:
        second_pass.append(msg)
        continue
    # Some messages contain multiple people, so split them out. 
    for thank in thanks:
        thank = thank.replace("to ", "")
        if "," in thank:
            thank = thank.split(",")
            security_thanks += [t.strip() for t in thank]
        elif "and" in thank:
            thank = thank.split("and")
            security_thanks += [t.strip() for t in thank]
        else:
            security_thanks.append(thank)

# inline cleanup
for i, thank in enumerate(security_thanks):
    security_thanks[i] = thank.split("for")[0]
    
for i, thank in enumerate(security_thanks):
    security_thanks[i] = thank.split("and")[0]
    
    
# remove duplicates
security_thanks = unique(security_thanks)
            
# parsing more complicated thanks, just stripping context. 
# outputs groups as a whole. More cleanup will be required.
for thank in second_pass: 
    thanks = ""
    found = False
    for line in thank.splitlines():
        if "Thanks" in line:
            found = True
        if found:
            thanks += line + " "
    security_thanks.append(thanks)

print("Security thanks:", len(security_thanks))
print("\n".join(security_thanks))

Security thanks: 64

Adam Johnson
Adam Zimmerman
Alan Crosswell
Alan Ryan
Allen Jonathan David
Andrew Chen Wang
Antonio Terceiro
Arsalan Ghassemi
Baptiste Mispelon
bcail
Ben Picolo
Benjamin Locher
Chris Bailey
Chris Lee
Claude Paroz
Daniel Swain
David Glenck
David Smith
David Wyde
Dennis Brinkrolf
Eugene Kovalev
Ferran Jovell
Florian Apolloner
Hervé Le Roy
Jacob Walls
Jakob Köhler
Johnny Metz
Jonny Park
Joseph Abrahams
Keryn Knight
Kevin Marsh
lind-marcus
Lucidot
Mariusz Felisiak
Matt Westcott
Michal Čihař
Motoyasu Saburi
Nick Pope
OutOfFocus4
Paolo Melchiorre
Paul in 't Hout
Pēteris Caune
Rick Yang
Sergey Fedoseev
Shai Berger
Silvio Gutierrez
Simon Charette
Sjoerd Job Postmus
Sourav Kumar
TakuYoshikai (Aeye Security Lab)
TengMA(@te3t123)
Terence Honles
Theodore Ni
Tim Graham
Tim McCurrach
Tobias Beng
Todor Velichkov
Tom Carrick
yakimka
אורי
Thanks Splunk team: Preston Elder, Jacob Davis, Jacob Moore, Matt Hanson, David Briggs, and a security researcher: Danylo Dmytriiev (DDV_UA) for t

_This list won't be as long as the original list, because we've already remove duplicates, and split out rows._

Instead of complicated regex, we could just use [NLTK](https://www.nltk.org/), no?

_Much of the following is based on https://unbiased-coder.com/extract-names-python-nltk/_

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')


In [43]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

def get_people(text): 
    """For a string of text, return people."""
    people = []
    nltk_results = ne_chunk(pos_tag(word_tokenize(text)))
    for nltk_result in nltk_results:
        if type(nltk_result) == Tree:
            name = ''
            for nltk_result_leaf in nltk_result.leaves():
                name += nltk_result_leaf[0] + ' '
            if nltk_result.label() == "PERSON":
                people.append(name.strip())
    return people

thanks_messages = git_commits_df[git_commits_df.message.str.contains("Thanks")]["message"].values.tolist()

people = []

for message in thanks_messages:
    people += get_people(message)

people = unique(people)

print("People:", len(people))
print("\n".join(people))

People: 86
Adam Johnson
Adam Zimmerman
Alan Crosswell
Alan Ryan
Andreas Pelme
Andrew Chen Wang
Antonio Terceiro
Arsalan Ghassemi
Baptiste Mispelon
Ben Picolo
Benjamin Locher
Carlton Gibson
Catalan
Changed DATE_FORMAT
Chris Jerdonek
Chris Lee
Claude Paroz
Danylo Dmytriiev
David Briggs
David Smith
Dennis Brinkrolf
Django
Exact
Ferran Jovell
Fixed
Fixed CheckConstraint
Fixed DoS
Fixed QuerySet
Florian Apolloner
Hannes Ljungberg
Hervé Le Roy
Jacob Davis
Jacob Moore
Jakob Köhler
Jonny Park
Joseph Abrahams
Keryn Knight
Kevin Marsh
Lucidot
Mariusz Felisiak
Matches
Matt Hanson
Matt Westcott
Michal Čihař
Month
Motoyasu Saburi
Moved
Ndlovu
Nick Pope
Paolo Melchiorre
Paul
Preston Elder
Python
Pēteris Caune
Redis
Refs
Rick Yang
Sergey Fedoseev
Shai Berger
Silvio Gutierrez
Simon Charette
Sjoerd Job Postmus
Sourav Kumar
Sphinx
Splunk
Switched
Terence Honles
Thanks Allen Jonathan David
Thanks Chris Bailey
Thanks Daniel Swain
Thanks David Glenck
Thanks David Wyde
Thanks Eugene Kovalev
Thanks Jacob Wal

There are some false negatives in there ("Django" is not a person), and some names that start with "Thanks", so we can remove those manually: 


In [44]:
for i, person in enumerate(people):
    
    # Trim out word set
    people[i] = person.replace("Thanks ", "")
    
    # Remove word set
    word_set = ["Fixed", "Changed", "Django", "Matches", "Redis", "Catalan", "Exact", 
                "Ref", "Sphinx", "Splunk", "Moved", "Month", "Python", "Switched"]
    for phrase in word_set:
        if phrase in person:
            people.pop(i)
    

people = unique(people)
print("People:", len(people))
print("\n".join(people))

People: 72
Adam Johnson
Adam Zimmerman
Alan Crosswell
Alan Ryan
Allen Jonathan David
Andreas Pelme
Andrew Chen Wang
Antonio Terceiro
Arsalan Ghassemi
Baptiste Mispelon
Ben Picolo
Benjamin Locher
Carlton Gibson
Changed DATE_FORMAT
Chris Bailey
Chris Jerdonek
Chris Lee
Claude Paroz
Daniel Swain
Danylo Dmytriiev
David Briggs
David Glenck
David Smith
David Wyde
Dennis Brinkrolf
Eugene Kovalev
Exact
Ferran Jovell
Fixed CheckConstraint
Fixed QuerySet
Florian Apolloner
Hannes Ljungberg
Hervé Le Roy
Jacob Davis
Jacob Moore
Jacob Walls
Jakob Köhler
Johnny Metz
Jonny Park
Joseph Abrahams
Keryn Knight
Kevin Marsh
Lucidot
Mariusz Felisiak
Matt Hanson
Matt Westcott
Michal Čihař
Motoyasu Saburi
Ndlovu
Nick Pope
OutOfFocus4
Paolo Melchiorre
Paul
Preston Elder
Pēteris Caune
Refs
Rick Yang
Sergey Fedoseev
Shai Berger
Silvio Gutierrez
Simon Charette
Sjoerd Job Postmus
Sourav Kumar
Splunk
Takuto Yoshikai
Terence Honles
Thibaud Colas
Tim Graham
Tim McCurrach
Tobias Bengfort
Todor Velichkov
Tom Carrick


But now we have a disperate set, compared to the original list. 


In [45]:
print("Regex:", len(security_thanks), "vs NLTK:", len(people))
for person in security_thanks:
    if person not in people:
        print(person)

Regex: 64 vs NLTK: 72

bcail
lind-marcus
Paul in 't Hout
TakuYoshikai (Aeye Security Lab)
TengMA(@te3t123)
Theodore Ni
Tobias Beng
yakimka
אורי
Thanks Splunk team: Preston Elder, Jacob Davis, Jacob Moore, Matt Hanson, David Briggs, and a security researcher: Danylo Dmytriiev (DDV_UA) for the report. 
Thanks to Adam Johnson, Claude Paroz, Keryn Knight, and Thibaud Colas for review. 
Thanks Florian Apolloner, Chris Jerdonek, Hannes Ljungberg, Nick Pope, and Mariusz Felisiak for reviews. 


So instead, let's just use NLTK on those thanks messages we originally deemed too complex. 


In [46]:
complex_thanks = []
people = []

# split out our thanks to parse and keep
for person in security_thanks:
    if "," in person:
        complex_thanks.append(person)
    else:
        people.append(person)
        
for thank in complex_thanks:
    people += get_people(thank)
    
people = unique(people)
for i, person in enumerate(people):
    if person in word_set:
        people.pop(i)

print("People:", len(people))
print("\n".join(people))

People: 70

Adam Johnson
Adam Zimmerman
Alan Crosswell
Alan Ryan
Allen Jonathan David
Andrew Chen Wang
Antonio Terceiro
Arsalan Ghassemi
Baptiste Mispelon
bcail
Ben Picolo
Benjamin Locher
Chris Bailey
Chris Jerdonek
Chris Lee
Claude Paroz
Daniel Swain
Danylo Dmytriiev
David Briggs
David Glenck
David Smith
David Wyde
Dennis Brinkrolf
Eugene Kovalev
Ferran Jovell
Florian Apolloner
Hannes Ljungberg
Hervé Le Roy
Jacob Davis
Jacob Moore
Jacob Walls
Jakob Köhler
Johnny Metz
Jonny Park
Joseph Abrahams
Keryn Knight
Kevin Marsh
lind-marcus
Lucidot
Mariusz Felisiak
Matt Hanson
Matt Westcott
Michal Čihař
Motoyasu Saburi
Nick Pope
OutOfFocus4
Paolo Melchiorre
Paul in 't Hout
Preston Elder
Pēteris Caune
Rick Yang
Sergey Fedoseev
Shai Berger
Silvio Gutierrez
Simon Charette
Sjoerd Job Postmus
Sourav Kumar
TakuYoshikai (Aeye Security Lab)
TengMA(@te3t123)
Terence Honles
Theodore Ni
Thibaud Colas
Tim Graham
Tim McCurrach
Tobias Beng
Todor Velichkov
Tom Carrick
yakimka
אורי


We'll use this version of as the list of people who contributed to security fixes. 

In [75]:
security_thanks = people

### Transifex

Another dataset we have is the transifex data. 

For each release, a new branch is created, for example https://github.com/django/django-docs-translations/tree/stable/4.1.x

If we use this repo, we can go through the commits since the last release and check for updates. 

However, these updates only include authors by year. 

For example, from a [recent commit](https://github.com/django/django-docs-translations/commit/ce809e91c8d8ade2de7982aa0014e9d1e77c1aa9#diff-7568ed75ebcd72f094cb0b97517e4f871aa26ba70271ada23ec71dd603273398R8):

```
-# Claude Paroz <claude@2xlibre.net>, 2013-2021
+# Claude Paroz <claude@2xlibre.net>, 2013-2022
```

This repo is based on output from a utility script [`manage_translations.py`](https://github.com/django/django-docs-translations/blob/stable/4.1.x/manage_translations.py) which itself relies on the package `transifex-client`.


`#TODO(glasnt) work out if the new transifex API can be used to pull authors in a better way`

A naive way of getting the translators for the branch is _very roughly_ map to the year that most development happened, and get any authors tagged with that year. 

Roughly: `git grep "^#"  | grep 2022 | cut -d"#" -f2 | cut -d',' -f1  | cut -d'<' -f1 | sort -n | uniq`

In [50]:
!git clone -b stable/{target_release}.x https://github.com/django/django-docs-translations

fatal: destination path 'django-docs-translations' already exists and is not an empty directory.


In [56]:
import re
from pathlib import Path
folder = Path.cwd() / "django-docs-translations"

year = "2022"

translators = []

for file in folder.glob("**/*.po"):
    with open(file) as f: 
        for line in f.readlines():
            if re.search(f"^#(.*){year}", line):
                translators.append(line.split(",")[0])

for i, author in enumerate(translators):
    translators[i] = author.replace("#","").strip().split("<")[0]
                
translator_thanks = unique(translators)


print("Translators:", len(translator_thanks))
print("\n".join(translator_thanks))

Translators: 42
0d21a39e384d88c2313b89b5042c04cb
Albert Lei
Bertalan Ivan
Claude Paroz
Darek505
decyma
e4db27214f7e7544f2022c647b585925_bb0e321
eitaro
Fery Setiawan
Giacomo M
Gyeongjun Paik
Hiroki Nakayama
Ikemoto Hideki
Jiawei Xu
Juna Salviati
Lu
Maciej Olko
Marco De Paoli
Marco Richetta
Mariusz Felisiak
Mirco Grillo
Nicola Guglielmi
Paolo Melchiorre
peiji liu
seungho
Stefan Ocetkiewicz
Taichi Taniguchi
Todd Kasaki
Tomaz Marcelino Cunha Neto
Toru Ohno
Veoco
Wu Xiangfeng
xiaqi516
yeongkwang
Youngjun Lee
Youxi Luo
汪心禾
龙虎义
문채원
이우현
이재준
정환 윤


# Analysis <a class="anchor" id="analysis"></a>
 
 
Our analysis is going to be based on our [assumptions](#assumptions). 

Some other assumptions from other projects: 

 * https://github.com/cncf/devstats/blob/master/README_K8s.md "Contribution: a review, comment, commit, issue or PR"

In [62]:
%%capture
import sys
!{sys.executable} -m pip install "rich[jupyter]"

and convert our lists to DataFrames

In [3]:
import pandas as pd

git_commits_df = pd.DataFrame(git_commits)
print("Git Commit keys:", ", ".join(list(git_commits_df.keys())))

git_trac_links_df = pd.DataFrame(git_trac_links)
pull_requests_df = pd.DataFrame(pull_requests)
print("Pull Request keys:", ", ".join(list(pull_requests_df.keys())))
pull_request_comments_df = pd.DataFrame(pr_comments)
print("Pull Request Comments keys:", ", ".join(list(pull_request_comments_df.keys())))
trac_tickets_df = pd.DataFrame(trac_tickets)
print("Trac Ticket keys:", ", ".join(list(trac_tickets_df.keys())))
trac_ticket_comments_df = pd.DataFrame(trac_ticket_comments)
print("Trac Ticket Comments keys:", ", ".join(list(trac_ticket_comments_df.keys())))

NameError: name 'git_commits' is not defined

For starters, who are our Git Commit Authors:

In [74]:
from collections import Counter

from rich.console import Console
from rich.table import Table
console = Console()


def print_table(table_name="Table", columns=["a", "b"], data=[[1,2],[3,4]], limit=None):
    table = Table(title=table_name)
    for col in columns:
        table.add_column(col)
    if limit:
        data = data[:limit]
        table.title += f" (limit {limit})"
    for x in data:
        table.add_row(*list([str(y) for y in x]))

    console.print(table)

    
gc_authors = Counter([g["author"] for g in git_commits])
print_table("Git Committers", ["Name", "Commits"], gc_authors.most_common(), limit=10)


   Git Committers (limit 10)   
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Name              ┃ Commits ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ Mariusz Felisiak  │ 205     │
│ Carlton Gibson    │ 60      │
│ Adam Johnson      │ 40      │
│ David Smith       │ 25      │
│ Nick Pope         │ 24      │
│ Jacob Walls       │ 24      │
│ Keryn Knight      │ 23      │
│ David Wobrock     │ 20      │
│ Claude Paroz      │ 19      │
│ Baptiste Mispelon │ 18      │
└───────────────────┴─────────┘

And who are our Pull Request Authors:

In [75]:
pr_authors = Counter([p["user"] for p in pull_requests])
print_table("Pull Request Authors", ["username", "Pull Requests"], pr_authors.most_common(), limit=10)

  Pull Request Authors (limit 10)  
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ username        ┃ Pull Requests ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ felixxm         │ 145           │
│ smithdc1        │ 37            │
│ carltongibson   │ 36            │
│ claudep         │ 27            │
│ jacobtylerwalls │ 20            │
│ adamchainz      │ 18            │
│ kezabelle       │ 18            │
│ mgaligniana     │ 16            │
│ charettes       │ 15            │
│ David-Wobrock   │ 15            │
└─────────────────┴───────────────┘

This is where things start to break, because we have Names and Aliases. 

Git works by `user.name` and `user.email` (you may be familiar with setting this up yourself in your local `git`).

GitHub works by way of usernames, and maps commits to users by the email addresses associated with the GitHub account.

You can use the GitHub API to get someone's GitHub display name from their email, but this may not match the name from their git commits. 

So, let's start by setting up some helper functions, then grabbing the data for all the users we know about. 

In [85]:
github_name = {}

def get_github_user(user):
    """For a GitHub username, get their name.
    NOTE(glasnt): debugging details have been added here to work out source of 
    missing data, rather than trying to use None or N/A, etc. 
    """
    try:
        data = github_api(f"/users/{user}")
    except ValueError:
        return user + " [not found on GitHub]"
    if data["name"] == None:
        return user + " [no GitHub name]"
    return data["name"]


users = [u["user"] for u in pull_requests] + [u["user"] for u in pr_comments] + [u["reporter"] for u in trac_tickets ] +[u["name"] for u in trac_ticket_comments]

for user in tqdm(unique(users)):
    github_name[user] = get_github_user(user)

  0%|          | 0/704 [00:00<?, ?it/s]

It'll be also useful later on to have a reversed dict, so we can either search for names by username, or usernames by name.

In [77]:
# Code golf :D
github_user = {v: k for k, v in github_name.items()}

print("GitHub users:", len(github_user))
print_table("GitHub Users", ["Username", "Name"], list(github_user.items()), limit=10)


GitHub users: 697


        GitHub Users (limit 10)        
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Username            ┃ Name          ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Joshua Cornutt      │ 01000101      │
│ Theofilos Alexiou   │ 2019342a      │
│ Adonys Alea Boffill │ aaboffill     │
│ Aapo Rista          │ aapris        │
│ Aymeric Augustin    │ aaugustin     │
│ Daniyal Abbasi      │ abbasidaniyal │
│ Abhyudai            │ abhiabhi94    │
│ Abhijeet            │ abhijeetviswa │
│ Aaron Chong         │ acjh          │
│ Adam Johnson        │ adamchainz    │
└─────────────────────┴───────────────┘

## Sampling by User

Using all this data, let's take some sample users and check all their contributions. 

Keeping in mind the set of data we have, and the mapping of identities 

```
git_commits author
pull_requests user->gname
pr_comments user->gname
trac_tickets reporter->gname*
trac_ticket_comments name->gname
security_thanks (name)
translators (name)
```

`*` Edge case: Trac accepts both GitHub logins and DjangoProject logins, so these may not match.

In [78]:
def user_contributions(user):
    
    if user not in github_name.keys():
        return "User not found."
    
    name = github_name[user]
    
    title=f"{name} ({user})"
    columns = ["Contribution", "Count"]
    data = []
    
    gcs = [g for g in git_commits if g["author"] == name]
    data.append(["Git Commits", str(len(gcs))])
    
    prs = [p for p in pull_requests if p["user"] == user]
    data.append(["Pull Requests Authored", str(len(prs))])
    
    prcs = [p for p in pr_comments if p["user"] == user]
    data.append(["Pull Request comments",  str(len(prcs))])
    
    tts = [t for t in trac_tickets if t["reporter"] == user]
    data.append(["Trac Tickets created",  str(len(tts))])
    
    ttcs = [t for t in trac_ticket_comments if t["name"] == user]
    data.append(["Trac Ticket comments",  str(len(ttcs))])
    
    st = [s for s in security_thanks if s == name]
    data.append(["Security thanks", str(len(st))])
    
    tt = [t for t in translator_thanks if t == name]
    data.append(["Translation thanks", str(len(tt))])
             
    print_table(title, columns, data)


# Mariusz is a Django Fellow, and has a contribution of every type in 4.1, so is a good test case.
user_contributions("felixxm")

# Carlton is also a Fellow, so probably has a bunch of data. 
user_contributions("carltongibson")

# And our author, just for fun.
user_contributions("glasnt")

    Mariusz Felisiak (felixxm)    
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Contribution           ┃ Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Git Commits            │ 205   │
│ Pull Requests Authored │ 145   │
│ Pull Request comments  │ 2163  │
│ Trac Tickets created   │ 27    │
│ Trac Ticket comments   │ 2908  │
│ Security thanks        │ 1     │
│ Translation thanks     │ 1     │
└────────────────────────┴───────┘

  Carlton Gibson (carltongibson)  
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Contribution           ┃ Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Git Commits            │ 60    │
│ Pull Requests Authored │ 36    │
│ Pull Request comments  │ 493   │
│ Trac Tickets created   │ 8     │
│ Trac Ticket comments   │ 520   │
│ Security thanks        │ 0     │
│ Translation thanks     │ 0     │
└────────────────────────┴───────┘

    Katie McLaughlin (glasnt)     
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Contribution           ┃ Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Git Commits            │ 0     │
│ Pull Requests Authored │ 0     │
│ Pull Request comments  │ 4     │
│ Trac Tickets created   │ 0     │
│ Trac Ticket comments   │ 0     │
│ Security thanks        │ 0     │
│ Translation thanks     │ 0     │
└────────────────────────┴───────┘

We can also collate all the contribution types by user. 

Important notes to remember:

 * Data for PRs and Trac Tickets can be mapped to a name from a username.
 * Names in thanks lists _may_ be able to be mapped to a username.
 * Git names _may_ be able to be mapped to a username. 

In [ ]:
contrib_array = {"g": 0, "pr": 0, "prc": 0, "ttr": 0, "ttc": 0, "st": 0, "tt": 0}

contributors = {}

def merge_contrib(records, key):
    for g in records:
        user, num = g
        if not user in contributors.keys():
            contributors[user] = contrib_array.copy()
        contributors[user][key] = num
    

gc = Counter([github_user.get(g["author"], g["author"]) for g in git_commits]).most_common()
merge_contrib(gc, "g")
    
prc = Counter([g["user"] for g in pull_requests]).most_common()
merge_contrib(prc, "pr")

prcsc = Counter([g["user"] for g in pr_comments]).most_common()
merge_contrib(prcsc, "prc")

ttrc = Counter([g["reporter"] for g in trac_tickets]).most_common()
merge_contrib(ttrc, "ttr")

ttcc = Counter([github_user.get(g["name"], g["name"]) for g in trac_ticket_comments]).most_common()
merge_contrib(ttcc, "ttc")

stc = Counter([github_user.get(g, g) for g in security_thanks]).most_common()
merge_contrib(stc, "st")

ttc = Counter([github_user.get(g, g) for g in translator_thanks]).most_common()
merge_contrib(ttc, "tt")
    
contributors

table = Table(title=f"Contributions to Django {target_release}")
for col in ["Name", "User", "Commits", "PRs", "PR Cs", "Trac", "Trac Cs", "Sec.", "Transl."]:
    table.add_column(col)

for c in contributors:
    table.add_row(c, github_name.get(c, c), *[str(s[1]) for s in contributors[c].items()] )
   
console.print(table)

# Results <a class="anchor" id="results"></a>

Now, with the data we have, we can now generate a unique list of all the humans who contributed. 

This is going to be based only on the data we already have, and is biased on all our prior assumptions. 

In [ ]:
django_contributors = unique([github_name.get(g, g) for g in unique(contributors.keys())])
print(f"Contributors to Django {target_release}:", len(django_contributors))
print("\n".join(django_contributors))

----

Feedback? Comments? Questions? Corrections? 

Please log an issue on the GitHub repo, or email me: katie `[at]` glasnt.com 